In [1]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=31d850be503983b8ce18f6739dbccabcbf21430bc412ec931d691155c15aa979
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [2]:
import os
import pandas as pd
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

bike = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/bike_dataset.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/orders.csv')
customers = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/customers.csv')
employees = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/employees.csv')
ds_2020 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/DS_JobsSalaries_2020.csv')
ds_2021 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SQL/DS_JobsSalaries_2021.csv')

COUNT: 모든 행 개수 카운트
- COUNT(*): NULL 포함
- COUNT(컬럼명): NULL 제외

In [3]:
# 날씨 데이터의 값이 NULL 값인 전체 데이터 추출
mysql("""

SELECT *
FROM bike
WHERE weathersit IS NULL
""")

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-02-19,spring,2011,Feb,11,0,Saturday,0,None,0.44,0.4394,0.16,0.5821,47,76,123


In [4]:
# 전체 컬럼 개수, weathersit 컬럼 개수 확인
mysql("""

SELECT COUNT(*) AS cnt
      ,COUNT(weathersit) AS weather_cnt
FROM bike
""")

,cnt,weather_cnt
0,17379,17378


In [5]:
# 2011년 3월 casual, registered 고객 전체 이용 건수
# SUM: 값의 합, COUNT: 행의 개수
mysql("""

SELECT SUM(casual) AS casual_sum
      ,SUM(registered) AS registered_sum
      ,COUNT(casual) AS cnt
FROM bike
WHERE yr = 2011
AND mnth = 'Mar'
""")

,casual_sum,registered_sum,cnt
0,12826,51219,730


In [6]:
# 가장 큰 causal의 이용건수와 가장 작은 registereed의 이용 건수 확인
mysql("""

SELECT MAX(casual) AS casual_max
      ,MIN(casual) AS registered_min
FROM bike
""")

,casual_max,registered_min
0,367,0


In [7]:
# 문자열에 대해서도 MAX, MIN 적용 가능
# weekday 컬럼의 max 함수 적용
mysql("""

SELECT MAX(weekday) AS max_week
FROM bike
""")

,max_week
0,Wednesday


집계함수 특징
- 집계함수 내부에 조건문 사용 가능
- 집계함수의 결과값끼리 계산 가능

In [8]:
# 봄 기간의 데이터 카운트
mysql("""

SELECT COUNT(season) AS '봄기간데이터개수'
FROM bike
WHERE season = 'spring'
""")

,봄기간데이터개수
0,4242


In [9]:
# 휴일 기간 카운트
mysql("""

SELECT COUNT(CASE WHEN weekday = 'Sunday' THEN '휴일'
             WHEN weekday = 'Saturday' THEN '휴일'
             ELSE NULL
        END) AS '휴일기간'
FROM bike
""")

,휴일기간
0,5014


GROUPBY
- SELECT 절에 컬럼명 같이 기재해주기

In [10]:
# temp의 평균온도 확인
mysql("""

SELECT AVG(temp) AS "평균온도"
FROM bike

""")

,평균온도
0,0.496987


In [11]:
# 계절과 온도의 RAW 데이터를 확인
mysql("""

SELECT season, temp
FROM bike
""")

,season,temp
0,spring,0.24
1,spring,0.22
2,spring,0.22
3,spring,0.24
4,spring,0.24
...,...,...
17374,spring,0.26
17375,spring,0.26
17376,spring,0.26
17377,spring,0.26


In [12]:
# 계절별 평균 온도 확인
mysql("""

SELECT season AS "계절", AVG(temp) AS "평균온도"
FROM bike
GROUP BY season

""")

,계절,평균온도
0,fall,0.706410
1,spring,0.299147
2,summer,0.544663
3,winter,0.423138


In [13]:
# 일자 이용건수의 합계
# 1: SELECT 절의 첫 번째 컬럼
mysql("""

SELECT dteday AS "일자", SUM(cnt) AS "이용건수합계"
FROM bike
GROUP BY 1

""")

,일자,이용건수합계
0,2011-01-01,985
1,2011-01-02,801
2,2011-01-03,1349
3,2011-01-04,1562
4,2011-01-05,1600
...,...,...
726,2012-12-27,2114
727,2012-12-28,3095
728,2012-12-29,1341
729,2012-12-30,1796


In [14]:
# 날씨별 이용건수의 합계

mysql("""

SELECT weathersit AS "날씨", SUM(cnt) AS "이용건수합계"
FROM bike
GROUP BY 1

""")

,날씨,이용건수합계
0,None,123
1,Clear,2338050
2,Cloudy,795952
3,LightRain,158331
4,Snow_Thunderstorm,223


**작성순서**

SELECT

FROM

WHERE

GROUP BY

HAVING

ORDER BY

LIMIT

In [15]:
# 날씨별 이용건수의 합계 중 10만건 이상의 데이터
mysql("""

SELECT weathersit AS "날씨", SUM(cnt) AS "이용건수합계"
FROM bike
GROUP BY 1
HAVING SUM(cnt) >= 100000

""")

,날씨,이용건수합계
0,Clear,2338050
1,Cloudy,795952
2,LightRain,158331


WHERE
- 집계함수 사용 불가
- GROUP BY 전에 사용

HAVING
- 그룹에 적용
- GROUPBY 후에 사용

In [16]:
# 온도가 0.3 이상인 계절과 온도 데이터를 필터링
mysql("""

SELECT season AS "계절", temp AS "온도"
FROM bike
WHERE temp >= 0.3

""")

,계절,온도
0,spring,0.32
1,spring,0.38
2,spring,0.36
3,spring,0.42
4,spring,0.46
...,...,...
14501,spring,0.30
14502,spring,0.30
14503,spring,0.30
14504,spring,0.30


In [17]:
# 계절별 평균 온도가 0.3인 데이터 필터링
mysql("""

SELECT season AS "계절", AVG(temp) AS "평균온도"
FROM bike
GROUP BY 1
HAVING AVG(temp) >= 0.3

""")

,계절,평균온도
0,fall,0.706410
1,summer,0.544663
2,winter,0.423138
